In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch


Import the model

In [3]:
#Cargamos el modelo que vamos usar con la cantidad apropiada de num_labels(5 en nuestro caso)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

id2label = {
    0: "muy negativo",
    1: "negativo",
    2: "neutral",
    3: "positivo",
    4: "muy positivo"
}

label2id = {
    "muy negativo": 0,
    "negativo": 1,
    "neutral": 2,
    "positivo": 3,
    "muy positivo": 4
}


# Load the pre-trained model with the mappings
model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased", 
    num_labels=5,
    id2label=id2label,
    label2id=label2id
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#Importamos nuestros datos procesados
df = pd.read_excel("analsisDeSentimientoRedes_1_enero.xlsx")

train_text = df["text"].tolist()
train_label  = df["sentiment"].map(label2id).tolist()

max_len = len(max(train_text, key=len))


In [5]:
#Creamos una clase para nuestros datos donde los tokenizamos para el modelo BETO

from torch.utils.data import DataLoader, Dataset
from transformers import DataCollatorWithPadding

class DataSetX(Dataset):

    def __init__(self,text,labels,tokenizer,max_length = max_len):
        self.texts = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
            # Tokenize the input text
            encoding = self.tokenizer(
                self.texts[idx],
                truncation=True,
                padding="max_length",
                max_length=self.max_length,
                return_tensors="pt"
            )

            # Return a dictionary
            item = {key: val.squeeze() for key, val in encoding.items()}
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
            return item

"""
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")
        return {key: val.squeeze() for key, val in encoding.items()}, label
"""
    
    

#We transform our data in to a type suited for the Pytorch data pipeline
train_dataset = DataSetX(train_text, train_label, tokenizer)
#We use HF collator to padd the data to make sure it has the same input lenght
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#We define Pytorch's Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=data_collator) # Output Class: <class 'torch.utils.data.dataloader.DataLoader'>


In [70]:
for batch in train_dataloader:
    print(batch.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [74]:
for batch in train_dataloader:
    outputs = model(**batch)
    logits = outputs.logits
    print(logits)

tensor([[-0.8718,  0.6004, -1.3210,  2.5397, -1.8669],
        [-0.9049,  2.2967, -0.2428,  0.9813, -2.3972],
        [-0.7432,  0.0624, -0.9997,  2.5365, -1.4038],
        [-0.9699,  2.4953, -0.2270,  0.9113, -2.2876],
        [-0.7338,  0.1112, -0.8837,  2.6707, -1.5164],
        [-0.8219,  0.4070, -1.2007,  2.5788, -1.5895],
        [-1.0213,  2.6552, -0.2519,  0.6637, -2.3806],
        [-0.8472,  0.1938, -0.9464,  2.6010, -1.6008],
        [-0.7973,  2.5231, -0.5154,  0.1415, -2.0371],
        [-0.7337,  0.2403, -1.0289,  2.6004, -1.5602]],
       grad_fn=<AddmmBackward0>)
tensor([[-1.0701,  2.5495, -0.0358,  0.4475, -2.1845],
        [-0.9241,  2.5113, -0.1966,  0.5158, -2.3121],
        [-1.0359,  2.5425, -0.8138,  0.8246, -2.0271],
        [-0.2814, -0.0095,  1.7380,  0.1077, -0.9427],
        [-0.5518, -0.0756,  1.8198,  0.3252, -1.4322],
        [-0.8778,  0.3060,  0.7469,  1.3851, -1.6909],
        [-0.5095,  0.2679,  1.7367,  0.5594, -1.7005],
        [-0.8461,  2.8098, -0.3

In [52]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler

num_epochs = 3
# Define optimizer, loss function, and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()
num_training_steps = len(train_dataloader) * num_epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # for every key(k) and value(v) in the dictionary, we transfer them to the apropriate device
        batch = {k: v.to(device) for k, v in batch.items()}
        #Saves the labels of our current batch
        labels = batch["labels"]

        outputs = model(**batch)
        logits = outputs.logits
        loss = loss_fn(logits, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs} completed.")


Epoch 1/3 completed.
Epoch 2/3 completed.
Epoch 3/3 completed.


In [53]:
# Save
output_dir = "./trained_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to ./trained_model


In [6]:
# Load
from transformers import AutoTokenizer, AutoModelForSequenceClassification
output_dir = "./trained_model"

loaded_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(output_dir)
print("Model and tokenizer successfully loaded!")


Model and tokenizer successfully loaded!


In [7]:
def predict_sentiment(model_dir, text):
    """
    Load a trained model and tokenizer from a directory and predict sentiment for a given text.
    
    Args:
        model_dir (str): Directory where the trained model and tokenizer are saved.
        text (str): The input text to analyze.
    
    Returns:
        dict: A dictionary containing the sentiment category and associated probabilities.
    """
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform the prediction
    with torch.no_grad():
        outputs = model(**inputs)
        #change the output to a probability distribution
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # Map the predicted class to its label
    id2label = {0: 'muy negativo', 1: 'negativo', 2: 'neutral', 3: 'positivo', 4: 'muy positivo'}
    sentiment = id2label[predicted_class]
    
    # Return the result as a dictionary
    return {
        "sentiment": sentiment,
        "probabilities": probabilities.squeeze().tolist()
    }

In [9]:
model_directory = "./trained_model"
sample_text = "Las pymes son malas"
result = predict_sentiment(model_directory, sample_text)
print(result)

{'sentiment': 'negativo', 'probabilities': [0.08135326951742172, 0.37779924273490906, 0.22624237835407257, 0.2728400230407715, 0.04176516458392143]}
